In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

In [3]:
# Set the path to the data
data_dir = '/content/drive/My Drive/SRI'
csv_file = '/content/drive/My Drive/SRI/english.csv'


In [4]:
# Load the CSV file
data = pd.read_csv(csv_file)

data

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0
...,...,...
3405,Img/img062-051.png,z
3406,Img/img062-052.png,z
3407,Img/img062-053.png,z
3408,Img/img062-054.png,z


In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data['image'], data['label'], test_size=0.2, random_state=42)


In [6]:
# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data.loc[X_train.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 2728 validated image filenames belonging to 62 classes.


In [7]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)


Found 682 validated image filenames belonging to 62 classes.


In [8]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(400, 300, 3))

94765736/94765736 [==============================] - 2s 0us/step


In [9]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(62, activation='softmax')(x)

In [11]:
model = Model(inputs=base_model.input, outputs=output)


In [12]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
# Train the model
model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
86/86 [==============================] - 1616s 19s/step - loss: 3.9613 - accuracy: 0.0704 - val_loss: 3.6293 - val_accuracy: 0.1334
Epoch 2/20
86/86 [==============================] - 62s 722ms/step - loss: 3.3912 - accuracy: 0.1723 - val_loss: 3.2114 - val_accuracy: 0.2141
Epoch 3/20
86/86 [==============================] - 61s 711ms/step - loss: 3.0381 - accuracy: 0.2544 - val_loss: 2.9748 - val_accuracy: 0.2874
Epoch 4/20
86/86 [==============================] - 61s 712ms/step - loss: 2.7905 - accuracy: 0.3295 - val_loss: 2.7712 - val_accuracy: 0.3094
Epoch 5/20
86/86 [==============================] - 61s 711ms/step - loss: 2.5901 - accuracy: 0.3930 - val_loss: 2.5562 - val_accuracy: 0.3812
Epoch 6/20
86/86 [==============================] - 62s 716ms/step - loss: 2.4350 - accuracy: 0.4058 - val_loss: 2.4537 - val_accuracy: 0.3827
Epoch 7/20
86/86 [==============================] - 61s 711ms/step - loss: 2.2904 - accuracy: 0.4751 - val_loss: 2.3095 - val_accuracy: 0.4531